In [1]:
# 这部分的目标是演示如何给agent加上记忆功能，包含清空、查看、截断等实用功能
# ──────────────────────────────────────────────╮
# │ 1. 加载 Qwen API Key                         │
# │ （确保 .env 中有 QWEN_API_KEY）              │
# ╰──────────────────────────────────────────────╯

%load_ext dotenv
%dotenv

import os
from dotenv import load_dotenv
load_dotenv()

QWEN_API_KEY = os.getenv("QWEN_API_KEY")
if not QWEN_API_KEY:
    raise ValueError("❌ 请在 .env 文件中配置 QWEN_API_KEY")

print("✅ Qwen API Key 已加载！")

✅ Qwen API Key 已加载！


In [ ]:
# ╭──────────────────────────────────────────────╮
# │ 2. 实现 QwenAgentWithMemory                  │
# │                                              │
# │ 核心思想：                                   │
# │   - 用 self.messages 存储对话历史            │
# │   - 每次 chat() 都把新消息追加进去           │
# │   - 调用模型时传入完整历史                   │
# ╰──────────────────────────────────────────────╯

from openai import OpenAI

class QwenAgentWithMemory:
    def __init__(self, api_key: str, model: str = "qwen-max", max_messages: int = 20):
        """
        初始化带记忆的 Qwen Agent
        
        Args:
            api_key: 阿里云百炼 API Key
            model: Qwen 模型 ID（默认 qwen-max）
            max_messages: 最大保存消息数（防上下文过长）
        """
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
        self.model = model
        self.messages = []          # ← 记忆就在这里！
        self.max_messages = max_messages
    
    def chat(self, user_input: str) -> str:
        """与 Agent 对话（自动记录上下文）"""
        # 1. 添加用户消息
        self.messages.append({"role": "user", "content": user_input})

        # 在第1步添加用户消息后，预留空间：本次对话将新增 2 条（user + assistant）
        # 所以当前消息数最多只能到 self.max_messages - 2
        # 2. 自动截断过长历史（保留最近 max_messages 条）
        if len(self.messages) >= self.max_messages - 1:
            # 保留最近 (max_messages - 2) 条，为新对话腾出空间
            self.messages = self.messages[-(self.max_messages - 2):]
        
        # 3. 调用 Qwen
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=0.7
        ).choices[0].message.content
        
        # 4. 保存助手回复
        self.messages.append({"role": "assistant", "content": response})
        return response
    
    
    def clear_memory(self):
        """清空所有对话记忆"""
        self.messages = []
        print("🧠 记忆已清空！")
    
    def show_memory(self):
        """打印当前对话历史"""
        if not self.messages:
            print("📭 当前无记忆")
            return
        
        print(f"📜 当前记忆（共 {len(self.messages)} 条消息）：")
        for i, msg in enumerate(self.messages, 1):
            role = "👤 用户" if msg["role"] == "user" else "🤖 Qwen"
            # 截断长文本便于阅读
            content = (msg['content'][:70] + '...') if len(msg['content']) > 70 else msg['content']
            print(f"{i:2d}. {role}: {content}")

In [3]:
# ╭──────────────────────────────────────────────╮
# │ 3. 测试：Agent 能记住名字吗？                │
# ╰──────────────────────────────────────────────╯

# 初始化 Agent
agent = QwenAgentWithMemory(QWEN_API_KEY)

# 第一轮：自我介绍
print("💬 第一轮：")
resp1 = agent.chat("你好！我叫李华，是一名高中生。")
print("🤖:", resp1)

# 第二轮：测试记忆
print("\n💬 第二轮（关键测试）：")
resp2 = agent.chat("你还记得我的名字和身份吗？")
print("🤖:", resp2)

# 查看内部记忆
print("\n" + "="*50)
agent.show_memory()

💬 第一轮：
🤖: 你好，李华！很高兴认识你。作为一名高中生，你一定有很多有趣的事情可以分享吧。无论是学习上的挑战、未来的规划还是兴趣爱好，如果你有任何问题或需要建议，我都很乐意帮助。有什么特别想聊的话题吗？

💬 第二轮（关键测试）：
🤖: 当然记得，李华！你是一名高中生。如果你有任何问题或需要帮助的地方，尽管告诉我。无论是学习上的难题、未来的规划还是其他任何方面，我都很乐意为你提供帮助。

📜 当前记忆（共 4 条消息）：
 1. 👤 用户: 你好！我叫李华，是一名高中生。
 2. 🤖 Qwen: 你好，李华！很高兴认识你。作为一名高中生，你一定有很多有趣的事情可以分享吧。无论是学习上的挑战、未来的规划还是兴趣爱好，如果你有任何问题或需...
 3. 👤 用户: 你还记得我的名字和身份吗？
 4. 🤖 Qwen: 当然记得，李华！你是一名高中生。如果你有任何问题或需要帮助的地方，尽管告诉我。无论是学习上的难题、未来的规划还是其他任何方面，我都很乐意为你...


In [4]:
# ╭──────────────────────────────────────────────╮
# │ 4. 测试：多轮对话 + 自动截断                 │
# ╰──────────────────────────────────────────────╯

print("🔄 模拟 15 轮快速问答...")
for i in range(15):
    question = f"问题 {i+1}: 1+{i} 等于多少？"
    answer = agent.chat(question)
    print(f"Q{i+1}: {question} → A: {answer}")

print(f"\n📌 当前记忆条数: {len(agent.messages)}（应 ≤ 20）")
print("（由于每轮包含 user + assistant，实际最多 20 条 = 10 轮完整对话）")

# 清空记忆，准备下个测试
agent.clear_memory()

🔄 模拟 15 轮快速问答...
Q1: 问题 1: 1+0 等于多少？ → A: 1 + 0 等于 1。这是一个基本的加法运算。如果你还有其他数学问题或需要进一步的帮助，随时告诉我！
Q2: 问题 2: 1+1 等于多少？ → A: 1 + 1 等于 2。这是另一个基本的加法运算。如果你有更多数学问题或其他方面的疑问，随时告诉我！
Q3: 问题 3: 1+2 等于多少？ → A: 1 + 2 等于 3。这也是一个基本的加法运算。如果你还有其他数学问题或需要帮助的地方，随时告诉我！
Q4: 问题 4: 1+3 等于多少？ → A: 1 + 3 等于 4。这是另一个基本的加法运算。如果你有更多数学问题或其他方面的疑问，随时告诉我！
Q5: 问题 5: 1+4 等于多少？ → A: 1 + 4 等于 5。这也是一个基本的加法运算。如果你还有其他数学问题或需要帮助的地方，随时告诉我！
Q6: 问题 6: 1+5 等于多少？ → A: 1 + 5 等于 6。这是另一个基本的加法运算。如果你有更多数学问题或其他方面的疑问，随时告诉我！
Q7: 问题 7: 1+6 等于多少？ → A: 1 + 6 等于 7。这也是一个基本的加法运算。如果你还有其他数学问题或需要帮助的地方，随时告诉我！
Q8: 问题 8: 1+7 等于多少？ → A: 1 + 7 等于 8。这是另一个基本的加法运算。如果你有更多数学问题或其他方面的疑问，随时告诉我！
Q9: 问题 9: 1+8 等于多少？ → A: 1 + 8 等于 9。这也是一个基本的加法运算。如果你还有其他数学问题或需要帮助的地方，随时告诉我！
Q10: 问题 10: 1+9 等于多少？ → A: 1 + 9 等于 10。这是另一个基本的加法运算。如果你有更多数学问题或其他方面的疑问，随时告诉我！
Q11: 问题 11: 1+10 等于多少？ → A: 1 + 10 等于 11。这也是一个基本的加法运算。如果你还有其他数学问题或需要帮助的地方，随时告诉我！
Q12: 问题 12: 1+11 等于多少？ → A: 1 + 11 等于 12。这是另一个基本的加法运算。如果你有更多数学问题或其他方面的疑问，随时告诉我！
Q13: 问题 13: 1+12 等于多少？ → A: 1 + 12 等于 13。这是另一个基本的加法运算。如果你有更多数学问题

In [ ]:
# ╭──────────────────────────────────────────────╮
# │ 5. 总结                                      │
# ╰──────────────────────────────────────────────╯

print("🎉 你已成功实现：")
print("✅ 带记忆的 Qwen Agent")
print("✅ 自动上下文管理（追加 + 截断）")
print("✅ 清空/查看记忆功能")

print("\n💡 关键洞察：")
print("• Agent 的‘智能’不仅来自模型，更来自对上下文的管理")
print("• 所有高级功能（工具调用、推理链）都建立在记忆基础之上")

print("\n➡️ 下一步：我们将为 Agent 添加「工具」能力，")
print("   让它能查时间、做计算、访问外部信息！")

print("\n📚 提示：")
print("• 如需支持其他模型（如 Doubao），可参考 03/04 的统一接口设计")
print("• 当前实现已足够用于大多数单模型应用场景")